# 7.1. Writing Functions That Accept Any Number of Arguments

In [2]:
# accept any numbers of positional arguments 
def avg(first, *rest): # rest is a tuple of all the extra positional arguments passed
    return (first + sum(rest)) / (1 + len(rest))

# Sample use
avg(1,2)
avg(1,2,3,4,5)

# accept any number of keyword arguments 
import html

def make_element(name, value, **attrs): # attrs is a sictionary that the passed keyword arguments 
    keyvals = [' %s= "%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
        name=name,
        attrs=attr_str,
        value=html.escape(value))
    return element

# Example 
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)

# Creates '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

# accept both any number of positional and keyword-only arguments, use * and ** together 
def anyargs(*args, **kwargs):
    print(args) # A tuple 
    print(kwargs) # A dict


'<p>&lt;spam&gt;</p>'

# 7.2. Writing Functions That Only Accept Keyword Arguments

In [5]:
def recv(maxsize, *, block):
    'Receives a message'
    pass 

recv(1024, block=True)

def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m 
minimum(1, 5, 2, -5, 10)
minimum(1,5,2,-5,10, clip=0)

0

# 7.3. Attaching Informational Metadata to Function Arguments

In [7]:
# Function argument annotations can be a useful way to give programmers hints about how a function is supposed to be used.
def add(x:int, y:int) -> int:
    return x+y

help(add)
add.__annotations__

Help on function add in module __main__:

add(x:int, y:int) -> int
    # Function argument annotations can be a useful way to give programmers hints about how a function is supposed to be used.



{'return': int, 'x': int, 'y': int}

# 7.4. Returning Multiple Values from a Function

In [8]:
# return multiple values form a function, simply return a tuple 
def myfun():
    return 1,2,3

a, b, c = myfun()
print(a,b,c)

1 2 3


# 7.5. Defining Functions with Default Arguments

In [13]:
def spam(a, b=42):
    print(a, b)

spam(1)
spam(1,2)
# If the default value is supposed to be a mutable container
#Using a list as a default value 
def spam(a, b=None):
    if b is None:
        b = []
# whether an optional argument was given 
_no_value = object()
def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')

spam(1)
spam(1, 2)
spam(1, None)

1 42
1 2
No b value supplied


# 7.6. Defining Anonymous or Inline Functions

In [15]:
add = lambda x,y: x + y
add(2,3)
names = ['Davuid Beazley', 'Brain Jones', 'Raymod Hettinger']
sorted(names, key=lambda name: name.split()[-1].lower())


['Davuid Beazley', 'Raymod Hettinger', 'Brain Jones']

# 7.7 Capturing Variables in Anonymous Functions

In [20]:
x = 10
a = lambda y: x + y
x = 20 
b = lambda y: x + y
a(10)
b(10)

# lambda expression is a free variable that gets bound at runtime, not defineition time

funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))
    
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

4
4
4
4
4
0
1
2
3
4


# 7.8. Making an N-Argument Callable Work As a Callable with Fewer Arguments

In [25]:
def spam(a, b, c, d):
    print(a, b, c, d)

from functools import partial
s1 = partial(spam, 1)  # partial() fixes the values for certain arguments and returns a new callable as a result
s1(2, 3, 4)
s1(4,5,6)

s2 = partial(spam, d=42)
s2(1,2,3)

points = [(1,2),(3,4),(5,6),(7,8)]
import math 
def distance(p1, p2):
    x1, y1 = p1 
    x2, y2 = p2 
    return math.hypot(x2 - x1, y2 - y1)
pt = (4,3)
points.sort(key=partial(distance, pt))

def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

# A sample function 
def add(x, y):
    return x + y 

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial 
    
    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')
    
    p = Pool()
    p.apply_async(add, (3,4), callback=partial(output_result, log=log))
    p.close()
    p.join()

1 2 3 4
1 4 5 6
1 2 3 42


DEBUG:test:Got: 7


In [ ]:
# simple echo server
from socketserver import StreamRequestHandler, TCPServer 

class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)

serv = TCPServer(('', 15000), EchoHandler)
serv.serve_forever()

# 7.9. Replacing Single Method Classes with Functions

In [ ]:
# fetch URLs
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template 
    
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

# Example use. Download stock data from yahoo 
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={name}&f={fields}')
for line in yahoo.open(names='IBM,APPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

def urltemplate(template): # Using an inner function or closure 
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Example use
yahoo = urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,APPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

# 7.10. Carrying Extra State with Callback Functions

In [7]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Invoke the callback with the result 
    callback(result)

def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y 

apply_async(add, (2, 3), callback=print_result)
apply_async(add, ('hello', 'world'), callback=print_result)

class ResultHandler:
    def __init__(self):
        self.sequence = 0
        
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

# use a closure to capture state 
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler 
handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

# coroutine to accomplish 
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

# use its send() method as the callback 
handler = make_handler()
next(handler) 
apply_async(add, (2, 3), callback=handler.send)
apply_async(add,('hello', 'world'), callback=handler.send)


Got: 5
Got: helloworld
[1] Got: 5
[2] Got: helloworld
[1] Got: 5
[2] Got: helloworld
[1] Got: 5
[2] Got: helloworld


# 7.11. Inlining Callbacking Functions

In [ ]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Incoke the callback with the result 
    callback(result)

